In [1]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import cv2
import numpy as np
import pandas as pd
import os
import CNN_encoder as cn
import functions as fc
import pickle
import random
import time
import importlib

# 讀取圖片位置

In [2]:
train_df = pd.read_csv('./images/train.txt', sep=' ', header=None)
test_df = pd.read_csv('./images/test.txt', sep=' ', header=None)
val_df = pd.read_csv('./images/val.txt', sep=' ', header=None)
train_paths, train_response = './images/' + train_df[0].to_numpy(), train_df[1].to_numpy()
test_paths, test_response = './images/' + test_df[0].to_numpy(), test_df[1].to_numpy()
val_paths, val_response = './images/' + val_df[0].to_numpy(), val_df[1].to_numpy()
train_paths.shape, test_paths.shape, val_paths.shape

((63325,), (450,), (450,))

# 建立 model, loss, opt 物件

In [3]:
importlib.reload(fc)
importlib.reload(cn)

<module 'CNN_encoder' from '/home/re6091020/soa2/deep_learning/hw2/CNN_encoder.py'>

In [4]:
# 訓練用參數
batch_size = 20
epochs = 100
lr = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
model = cn.CNN_model_encoder().to(device)
lossf = nn.MSELoss().to(device)
opt = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

In [6]:
logger = fc.create_logger('./logger/', 'encoder_logger.txt')

In [7]:
len(list(model.parameters()))

30

# train encoder

In [8]:
# load model
# model = cn.CNN_model_encoder().to(device)
# model.load_state_dict(torch.load('./logger/CNN_model_encoder.pt'))

In [ ]:
start = time.time()
logger.info('start at {}'.format(start))

for epoch in range(1, epochs+1):
    img_index = 0
    img_len = train_paths.shape[0]
    while img_index < img_len:
        try:
            imgs = train_paths[img_index:img_index + batch_size]
            pic = fc.path2pic(imgs)
            inputs = fc.pic2tensor(pic).to(device)
            outputs = model(inputs)
            loss1 = lossf(outputs[1].detach(), outputs[6])
            loss2 = lossf(outputs[2].detach(), outputs[5])
            loss3 = lossf(outputs[8].detach(), outputs[13])
            loss4 = lossf(outputs[9].detach(), outputs[12])
            loss5 = lossf(outputs[0].detach(), outputs[14])

            opt.zero_grad()
            loss1.backward(retain_graph=True)
            loss2.backward(retain_graph=True)
            loss3.backward(retain_graph=True)
            loss4.backward(retain_graph=True)
            loss5.backward()
            opt.step()
            img_index += batch_size
            torch.save(model.state_dict(), './model/CNN_model_encoder.pt')
        except Exception as e:
            print(img_index)
            logger.info(e, exc_info=True)

    train_time = time.time() - start
    torch.save(model.state_dict(), './model/CNN_model_encoder.pt')
    logger.info('train time {}, {}/{}'.format(train_time, epoch, epochs))

2021-03-27 21:09:05,465 INFO start at 1616850545.4650593


# feature extract part for train machine learn

# for train data

# 提取圖片位置

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
logger = fc.create_logger('./', 'extract_logger.txt')

In [4]:
train_df = pd.read_csv('./images/train.txt', sep=' ', header=None)
test_df = pd.read_csv('./images/test.txt', sep=' ', header=None)
val_df = pd.read_csv('./images/val.txt', sep=' ', header=None)
train_paths, train_response = './images/' + train_df[0].to_numpy(), train_df[1].to_numpy()
test_paths, test_response = './images/' + test_df[0].to_numpy(), test_df[1].to_numpy()
val_paths, val_response = './images/' + val_df[0].to_numpy(), val_df[1].to_numpy()
train_paths.shape, test_paths.shape, val_paths.shape

((63325,), (450,), (450,))

In [5]:
# load model
model = cn.CNN_model_encoder().to(device)
model.load_state_dict(torch.load('./model/CNN_model_encoder.pt'))

<All keys matched successfully>

In [6]:
batch_size = 20
img_index = 0
img_len = val_paths.shape[0]
features = []

while img_index < img_len:
    imgs = val_paths[img_index:img_index + batch_size]
    pic = fc.path2pic(imgs)
    inputs = fc.pic2tensor(pic).to(device)
    outputs = model.feature_extract(inputs)
    img_index += batch_size
    features.append(outputs)
    logger.info('img_index:{}'.format(img_index))
feature_array = torch.cat(features, dim=0)
feature_array_cpu = feature_array.clone().detach().cpu()
torch.save(feature_array_cpu, './imgFeature/valFeature.pt')
del(feature_array)
del(feature_array_cpu)

2021-03-28 14:48:43,432 INFO img_index:20
2021-03-28 14:48:45,217 INFO img_index:40
2021-03-28 14:48:46,744 INFO img_index:60
2021-03-28 14:48:48,383 INFO img_index:80
2021-03-28 14:48:49,968 INFO img_index:100
2021-03-28 14:48:51,656 INFO img_index:120
2021-03-28 14:48:53,305 INFO img_index:140
2021-03-28 14:48:54,893 INFO img_index:160
2021-03-28 14:48:56,410 INFO img_index:180
2021-03-28 14:48:57,928 INFO img_index:200
2021-03-28 14:48:59,448 INFO img_index:220
2021-03-28 14:49:00,968 INFO img_index:240
2021-03-28 14:49:02,444 INFO img_index:260
2021-03-28 14:49:03,977 INFO img_index:280
2021-03-28 14:49:05,539 INFO img_index:300
2021-03-28 14:49:07,053 INFO img_index:320
2021-03-28 14:49:08,566 INFO img_index:340
2021-03-28 14:49:10,099 INFO img_index:360
2021-03-28 14:49:11,612 INFO img_index:380
2021-03-28 14:49:13,143 INFO img_index:400
2021-03-28 14:49:14,612 INFO img_index:420
2021-03-28 14:49:16,164 INFO img_index:440
2021-03-28 14:49:16,908 INFO img_index:460


In [ ]:
batch_size = 20
img_index = 0
img_len = test_paths.shape[0]
features = []

while img_index < img_len:
    imgs = test_paths[img_index:img_index + batch_size]
    pic = fc.path2pic(imgs)
    inputs = fc.pic2tensor(pic).to(device)
    outputs = model.feature_extract(inputs)
    img_index += batch_size
    features.append(outputs)
    logger.info('img_index:{}'.format(img_index))
feature_array = torch.cat(features, dim=0)
feature_array_cpu = feature_array.clone().detach().cpu()
torch.save(feature_array_cpu, './imgFeature/testFeature.pt')
del(feature_array)
del(feature_array_cpu)

In [7]:
batch_size = 20
img_index = 0
img_len = train_paths.shape[0]
features = []

while img_index < img_len:
    imgs = train_paths[img_index:img_index + batch_size]
    pic = fc.path2pic(imgs)
    inputs = fc.pic2tensor(pic).to(device)
    outputs = model.feature_extract(inputs)
    img_index += batch_size
    features.append(outputs)
    logger.info('img_index:{}'.format(img_index))
feature_array = torch.cat(features, dim=0)
feature_array_cpu = feature_array.clone().detach().cpu()
torch.save(feature_array_cpu, './imgFeature/trainFeature.pt')
del(feature_array)
del(feature_array_cpu)

2021-03-28 15:53:15,307 INFO img_index:20
2021-03-28 15:53:17,103 INFO img_index:40
2021-03-28 15:53:19,115 INFO img_index:60
2021-03-28 15:53:21,249 INFO img_index:80
2021-03-28 15:53:23,097 INFO img_index:100
2021-03-28 15:53:24,912 INFO img_index:120
2021-03-28 15:53:26,541 INFO img_index:140
2021-03-28 15:53:28,184 INFO img_index:160
2021-03-28 15:53:29,746 INFO img_index:180
2021-03-28 15:53:31,381 INFO img_index:200
2021-03-28 15:53:32,900 INFO img_index:220
2021-03-28 15:53:34,448 INFO img_index:240
2021-03-28 15:53:36,043 INFO img_index:260
2021-03-28 15:53:37,672 INFO img_index:280
2021-03-28 15:53:39,223 INFO img_index:300
2021-03-28 15:53:40,693 INFO img_index:320
2021-03-28 15:53:42,189 INFO img_index:340
2021-03-28 15:53:43,781 INFO img_index:360
2021-03-28 15:53:45,305 INFO img_index:380
2021-03-28 15:53:46,882 INFO img_index:400
2021-03-28 15:53:48,423 INFO img_index:420
2021-03-28 15:53:49,948 INFO img_index:440
2021-03-28 15:53:51,451 INFO img_index:460
2021-03-28 15:5

2021-03-28 15:58:00,587 INFO img_index:3780
2021-03-28 15:58:02,102 INFO img_index:3800
2021-03-28 15:58:03,579 INFO img_index:3820
2021-03-28 15:58:05,076 INFO img_index:3840
2021-03-28 15:58:06,578 INFO img_index:3860
2021-03-28 15:58:08,034 INFO img_index:3880
2021-03-28 15:58:09,544 INFO img_index:3900
2021-03-28 15:58:11,026 INFO img_index:3920
2021-03-28 15:58:12,542 INFO img_index:3940
2021-03-28 15:58:14,013 INFO img_index:3960
2021-03-28 15:58:15,484 INFO img_index:3980
2021-03-28 15:58:16,938 INFO img_index:4000
2021-03-28 15:58:18,447 INFO img_index:4020
2021-03-28 15:58:19,893 INFO img_index:4040
2021-03-28 15:58:21,337 INFO img_index:4060
2021-03-28 15:58:22,798 INFO img_index:4080
2021-03-28 15:58:24,278 INFO img_index:4100
2021-03-28 15:58:25,739 INFO img_index:4120
2021-03-28 15:58:27,208 INFO img_index:4140
2021-03-28 15:58:28,734 INFO img_index:4160
2021-03-28 15:58:30,194 INFO img_index:4180
2021-03-28 15:58:31,716 INFO img_index:4200
2021-03-28 15:58:33,227 INFO img

2021-03-28 16:02:38,321 INFO img_index:7520
2021-03-28 16:02:39,947 INFO img_index:7540
2021-03-28 16:02:41,471 INFO img_index:7560
2021-03-28 16:02:42,983 INFO img_index:7580
2021-03-28 16:02:44,527 INFO img_index:7600
2021-03-28 16:02:46,039 INFO img_index:7620
2021-03-28 16:02:47,540 INFO img_index:7640
2021-03-28 16:02:49,010 INFO img_index:7660
2021-03-28 16:02:50,456 INFO img_index:7680
2021-03-28 16:02:51,919 INFO img_index:7700
2021-03-28 16:02:53,379 INFO img_index:7720
2021-03-28 16:02:54,809 INFO img_index:7740
2021-03-28 16:02:56,322 INFO img_index:7760
2021-03-28 16:02:57,857 INFO img_index:7780
2021-03-28 16:02:59,353 INFO img_index:7800
2021-03-28 16:03:00,832 INFO img_index:7820
2021-03-28 16:03:02,326 INFO img_index:7840
2021-03-28 16:03:03,813 INFO img_index:7860
2021-03-28 16:03:05,263 INFO img_index:7880
2021-03-28 16:03:06,716 INFO img_index:7900
2021-03-28 16:03:08,232 INFO img_index:7920
2021-03-28 16:03:09,721 INFO img_index:7940
2021-03-28 16:03:11,187 INFO img

2021-03-28 16:07:15,735 INFO img_index:11220
2021-03-28 16:07:17,205 INFO img_index:11240
2021-03-28 16:07:18,701 INFO img_index:11260
2021-03-28 16:07:20,182 INFO img_index:11280
2021-03-28 16:07:21,678 INFO img_index:11300
2021-03-28 16:07:23,185 INFO img_index:11320
2021-03-28 16:07:24,660 INFO img_index:11340
2021-03-28 16:07:26,152 INFO img_index:11360
2021-03-28 16:07:27,625 INFO img_index:11380
2021-03-28 16:07:29,091 INFO img_index:11400
2021-03-28 16:07:30,578 INFO img_index:11420
2021-03-28 16:07:32,073 INFO img_index:11440
2021-03-28 16:07:33,565 INFO img_index:11460
2021-03-28 16:07:35,050 INFO img_index:11480
2021-03-28 16:07:36,510 INFO img_index:11500
2021-03-28 16:07:37,981 INFO img_index:11520
2021-03-28 16:07:39,443 INFO img_index:11540
2021-03-28 16:07:40,892 INFO img_index:11560
2021-03-28 16:07:42,358 INFO img_index:11580
2021-03-28 16:07:43,836 INFO img_index:11600
2021-03-28 16:07:45,317 INFO img_index:11620
2021-03-28 16:07:46,768 INFO img_index:11640
2021-03-28

2021-03-28 16:11:49,087 INFO img_index:14880
2021-03-28 16:11:50,639 INFO img_index:14900
2021-03-28 16:11:52,093 INFO img_index:14920
2021-03-28 16:11:53,614 INFO img_index:14940
2021-03-28 16:11:55,086 INFO img_index:14960
2021-03-28 16:11:56,540 INFO img_index:14980
2021-03-28 16:11:58,025 INFO img_index:15000
2021-03-28 16:11:59,499 INFO img_index:15020
2021-03-28 16:12:00,992 INFO img_index:15040
2021-03-28 16:12:02,447 INFO img_index:15060
2021-03-28 16:12:03,933 INFO img_index:15080
2021-03-28 16:12:05,407 INFO img_index:15100
2021-03-28 16:12:06,895 INFO img_index:15120
2021-03-28 16:12:08,369 INFO img_index:15140
2021-03-28 16:12:09,845 INFO img_index:15160
2021-03-28 16:12:11,318 INFO img_index:15180
2021-03-28 16:12:12,763 INFO img_index:15200
2021-03-28 16:12:14,217 INFO img_index:15220
2021-03-28 16:12:15,710 INFO img_index:15240
2021-03-28 16:12:17,197 INFO img_index:15260
2021-03-28 16:12:18,693 INFO img_index:15280
2021-03-28 16:12:20,187 INFO img_index:15300
2021-03-28

...

2021-03-28 16:53:10,807 INFO img_index:47820
2021-03-28 16:53:12,313 INFO img_index:47840
2021-03-28 16:53:13,830 INFO img_index:47860
2021-03-28 16:53:15,348 INFO img_index:47880
2021-03-28 16:53:16,850 INFO img_index:47900
2021-03-28 16:53:18,364 INFO img_index:47920
2021-03-28 16:53:19,892 INFO img_index:47940
2021-03-28 16:53:21,418 INFO img_index:47960
2021-03-28 16:53:22,889 INFO img_index:47980
2021-03-28 16:53:24,428 INFO img_index:48000
2021-03-28 16:53:25,924 INFO img_index:48020
2021-03-28 16:53:27,436 INFO img_index:48040
2021-03-28 16:53:28,939 INFO img_index:48060
2021-03-28 16:53:30,438 INFO img_index:48080
2021-03-28 16:53:31,927 INFO img_index:48100
2021-03-28 16:53:33,457 INFO img_index:48120
2021-03-28 16:53:34,989 INFO img_index:48140
2021-03-28 16:53:36,511 INFO img_index:48160
2021-03-28 16:53:38,002 INFO img_index:48180
2021-03-28 16:53:39,489 INFO img_index:48200
2021-03-28 16:53:40,982 INFO img_index:48220
2021-03-28 16:53:42,484 INFO img_index:48240
2021-03-28

2021-03-28 16:57:58,770 INFO img_index:51480
2021-03-28 16:58:00,476 INFO img_index:51500
2021-03-28 16:58:02,189 INFO img_index:51520
2021-03-28 16:58:03,938 INFO img_index:51540
2021-03-28 16:58:05,676 INFO img_index:51560
2021-03-28 16:58:07,475 INFO img_index:51580
2021-03-28 16:58:09,206 INFO img_index:51600
2021-03-28 16:58:10,949 INFO img_index:51620
2021-03-28 16:58:12,691 INFO img_index:51640
2021-03-28 16:58:14,373 INFO img_index:51660
2021-03-28 16:58:16,095 INFO img_index:51680
2021-03-28 16:58:17,822 INFO img_index:51700
2021-03-28 16:58:19,600 INFO img_index:51720
2021-03-28 16:58:21,361 INFO img_index:51740
2021-03-28 16:58:23,107 INFO img_index:51760
2021-03-28 16:58:24,780 INFO img_index:51780
2021-03-28 16:58:26,471 INFO img_index:51800
2021-03-28 16:58:28,115 INFO img_index:51820
2021-03-28 16:58:29,826 INFO img_index:51840
2021-03-28 16:58:31,529 INFO img_index:51860
2021-03-28 16:58:33,253 INFO img_index:51880
2021-03-28 16:58:34,935 INFO img_index:51900
2021-03-28

2021-03-28 17:02:56,496 INFO img_index:55140
2021-03-28 17:02:57,983 INFO img_index:55160
2021-03-28 17:02:59,457 INFO img_index:55180
2021-03-28 17:03:00,944 INFO img_index:55200
2021-03-28 17:03:02,417 INFO img_index:55220
2021-03-28 17:03:03,883 INFO img_index:55240
2021-03-28 17:03:05,330 INFO img_index:55260
2021-03-28 17:03:06,809 INFO img_index:55280
2021-03-28 17:03:08,309 INFO img_index:55300
2021-03-28 17:03:09,800 INFO img_index:55320
2021-03-28 17:03:11,283 INFO img_index:55340
2021-03-28 17:03:12,746 INFO img_index:55360
2021-03-28 17:03:14,280 INFO img_index:55380
2021-03-28 17:03:15,764 INFO img_index:55400
2021-03-28 17:03:17,431 INFO img_index:55420
2021-03-28 17:03:18,930 INFO img_index:55440
2021-03-28 17:03:20,398 INFO img_index:55460
2021-03-28 17:03:21,889 INFO img_index:55480
2021-03-28 17:03:23,350 INFO img_index:55500
2021-03-28 17:03:24,815 INFO img_index:55520
2021-03-28 17:03:26,245 INFO img_index:55540
2021-03-28 17:03:27,708 INFO img_index:55560
2021-03-28

2021-03-28 17:07:36,218 INFO img_index:58800
2021-03-28 17:07:37,738 INFO img_index:58820
2021-03-28 17:07:39,241 INFO img_index:58840
2021-03-28 17:07:40,749 INFO img_index:58860
2021-03-28 17:07:42,285 INFO img_index:58880
2021-03-28 17:07:43,859 INFO img_index:58900
2021-03-28 17:07:45,670 INFO img_index:58920
2021-03-28 17:07:47,448 INFO img_index:58940
2021-03-28 17:07:49,186 INFO img_index:58960
2021-03-28 17:07:50,933 INFO img_index:58980
2021-03-28 17:07:52,658 INFO img_index:59000
2021-03-28 17:07:54,388 INFO img_index:59020
2021-03-28 17:07:56,082 INFO img_index:59040
2021-03-28 17:07:57,802 INFO img_index:59060
2021-03-28 17:07:59,502 INFO img_index:59080
2021-03-28 17:08:01,223 INFO img_index:59100
2021-03-28 17:08:02,910 INFO img_index:59120
2021-03-28 17:08:04,627 INFO img_index:59140
2021-03-28 17:08:06,365 INFO img_index:59160
2021-03-28 17:08:08,141 INFO img_index:59180
2021-03-28 17:08:09,807 INFO img_index:59200
2021-03-28 17:08:11,575 INFO img_index:59220
2021-03-28

2021-03-28 17:12:27,404 INFO img_index:62460
2021-03-28 17:12:28,930 INFO img_index:62480
2021-03-28 17:12:30,432 INFO img_index:62500
2021-03-28 17:12:31,958 INFO img_index:62520
2021-03-28 17:12:33,474 INFO img_index:62540
2021-03-28 17:12:34,961 INFO img_index:62560
2021-03-28 17:12:36,516 INFO img_index:62580
2021-03-28 17:12:38,059 INFO img_index:62600
2021-03-28 17:12:39,602 INFO img_index:62620
2021-03-28 17:12:41,199 INFO img_index:62640
2021-03-28 17:12:42,745 INFO img_index:62660
2021-03-28 17:12:44,266 INFO img_index:62680
2021-03-28 17:12:45,803 INFO img_index:62700
2021-03-28 17:12:47,306 INFO img_index:62720
2021-03-28 17:12:48,835 INFO img_index:62740
2021-03-28 17:12:50,333 INFO img_index:62760
2021-03-28 17:12:51,872 INFO img_index:62780
2021-03-28 17:12:53,447 INFO img_index:62800
2021-03-28 17:12:54,971 INFO img_index:62820
2021-03-28 17:12:56,489 INFO img_index:62840
2021-03-28 17:12:58,016 INFO img_index:62860
2021-03-28 17:12:59,523 INFO img_index:62880
2021-03-28

In [4]:
trainFeature = torch.load('./imgFeature/trainFeature.pt').numpy()
testFeature = torch.load('./imgFeature/testFeature.pt').numpy()
valFeature = torch.load('./imgFeature/valFeature.pt').numpy()

In [5]:
with open('./imgFeature/trainFeature.pickle', 'wb') as f:
    pickle.dump(trainFeature, f)
with open('./imgFeature/testFeature.pickle', 'wb') as f:
    pickle.dump(testFeature, f)
with open('./imgFeature/valFeature.pickle', 'wb') as f:
    pickle.dump(valFeature, f)

# try

In [3]:
path_norm = './train/NORMAL/'
imglist_norm = os.listdir(path_norm)
path_pneu = './train/PNEUMONIA/'
imglist_pneu = os.listdir(path_pneu)
img_list = []
for i in imglist_norm:
    img_list.append(path_norm + i)
    
for i in imglist_pneu:
    img_list.append(path_pneu + i)
    
random.shuffle(img_list)
len(img_list)

5216

In [4]:
# 訓練用參數
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
model = CNN_model_encoder().to(device)
lossf = nn.MSELoss()

In [6]:
imgs = img_list[0:10]
pic = path2pic(imgs)
inputs = pic2tensor(pic).to(device)

In [7]:
inputs.shape

torch.Size([10, 3, 512, 512])

In [8]:
outputs = model(inputs)

In [11]:
loss5 = lossf(outputs[0].detach(), outputs[14])

In [15]:
loss5.backward(retain_graph=True)

In [21]:
loss1 = lossf(outputs[1].detach(), outputs[6])
loss2 = lossf(outputs[2].detach(), outputs[5])
loss3 = lossf(outputs[8].detach(), outputs[13])
loss4 = lossf(outputs[9].detach(), outputs[12])
loss5 = lossf(outputs[0].detach(), outputs[14])

In [22]:
loss1, loss2, loss3, loss4, loss5

(tensor(0.6279, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.3595, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.4133, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(1.4666, device='cuda:0', grad_fn=<MseLossBackward>),
 tensor(0.8438, device='cuda:0', grad_fn=<MseLossBackward>))

In [23]:
loss1.backward(retain_graph=True)

In [24]:
loss2.backward(retain_graph=True)

In [48]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
# m = nn.ConvTranspose2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
input = torch.randn(20, 16, 50, 100)
output = m(input)
# exact output size can be also specified as an argument
input = torch.randn(1, 16, 12, 12)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
h = downsample(input)
h.size()
torch.Size([1, 16, 6, 6])
output = upsample(h, output_size=input.size())
output.size()
torch.Size([1, 16, 12, 12])

torch.Size([1, 16, 12, 12])

In [52]:
# With square kernels and equal stride
m = nn.ConvTranspose2d(16, 33, 3, stride=1)
input = torch.randn(20, 16, 50, 100)

In [53]:
output = m(input)
output.shape

torch.Size([20, 33, 52, 102])

In [65]:
m = nn.Conv2d(3, 1, (3, 3), stride=1)

In [61]:
len(list(m.parameters()))

2

In [62]:
list(m.parameters())[0].shape, list(m.parameters())[1].shape

(torch.Size([8, 3, 3, 3]), torch.Size([8]))

In [66]:
input = torch.randn(20, 3, 50, 100)

In [68]:
m(input).shape

torch.Size([20, 1, 48, 98])

In [94]:
input = torch.rand(2, 1, 5, 5)
input, input.shape

(tensor([[[[0.3824, 0.2618, 0.8074, 0.4320, 0.8458],
           [0.4926, 0.1995, 0.1289, 0.6962, 0.2754],
           [0.2103, 0.5003, 0.0784, 0.1984, 0.2855],
           [0.9783, 0.1403, 0.6125, 0.6148, 0.9563],
           [0.8009, 0.4543, 0.0335, 0.1047, 0.0306]]],
 
 
         [[[0.2588, 0.2052, 0.7625, 0.1607, 0.3166],
           [0.3952, 0.2438, 0.9578, 0.8956, 0.9670],
           [0.6519, 0.7244, 0.5740, 0.9927, 0.9858],
           [0.4974, 0.8008, 0.7410, 0.8023, 0.8763],
           [0.3673, 0.3164, 0.7160, 0.3681, 0.2028]]]]),
 torch.Size([2, 1, 5, 5]))

In [95]:
m = nn.AdaptiveMaxPool2d((4, 4))
output = m(input)
output, output.shape

(tensor([[[[0.4926, 0.8074, 0.8074, 0.8458],
           [0.5003, 0.5003, 0.6962, 0.6962],
           [0.9783, 0.6125, 0.6148, 0.9563],
           [0.9783, 0.6125, 0.6148, 0.9563]]],
 
 
         [[[0.3952, 0.9578, 0.9578, 0.9670],
           [0.7244, 0.9578, 0.9927, 0.9927],
           [0.8008, 0.8008, 0.9927, 0.9927],
           [0.8008, 0.8008, 0.8023, 0.8763]]]]),
 torch.Size([2, 1, 4, 4]))

In [3]:
x = torch.rand(5, 3, 10, 10)

In [12]:
m = nn.LayerNorm(x.shape[2:], elementwise_affine=False)

In [13]:
output = m(x)

In [14]:
output

tensor([[[[ 0.5829,  1.4797,  0.4275,  ...,  0.0336, -0.5549, -0.4246],
          [ 0.1870, -0.1482,  0.5749,  ..., -1.0905, -1.6453,  0.7862],
          [ 0.6661, -0.2262, -1.0174,  ..., -1.4119,  1.2582, -0.2301],
          ...,
          [ 1.3639,  0.3099,  0.6002,  ..., -0.6457,  1.5809,  1.2038],
          [ 0.3407,  1.5708, -1.5265,  ...,  0.3097, -1.5203, -0.2271],
          [ 0.0590, -1.6453, -0.5070,  ...,  0.0652, -0.1709, -0.0558]],

         [[ 0.7759, -1.2816,  1.4900,  ...,  0.8405,  0.5381, -0.0728],
          [ 0.2517,  0.4695, -1.1645,  ...,  0.2120,  0.8396, -1.4304],
          [-0.7659, -0.2544,  0.8152,  ..., -0.2501,  0.8558, -0.1098],
          ...,
          [-0.3252,  1.3697, -0.7355,  ..., -1.3774,  1.4510, -0.4577],
          [ 1.3610, -0.1968, -0.1480,  ...,  0.0546, -0.2145,  1.2643],
          [-0.8530, -0.9829,  0.7633,  ...,  1.2993, -1.8078,  1.3453]],

         [[-1.3533, -0.2200,  1.4388,  ..., -0.4057, -1.2009,  0.2431],
          [ 0.5953, -0.6241,  